In [1]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# 1. Baca data
df = pd.read_excel("schizoCase.xlsx")

# 2. Pisahkan data training dan data uji
data_train = df.iloc[:-1, :]
data_test = df.iloc[-1:, :]

# 3. Ambil hanya kolom numerik
X_train = data_train.select_dtypes(include='number').iloc[:, :-1]
y_train = data_train.iloc[:, -1]
X_test = data_test.select_dtypes(include='number').iloc[:, :-1]

# 4. Standarisasi data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 5. Model SVM
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', probability=True, random_state=0)
svm_model.fit(X_train_scaled, y_train)

# 6. Prediksi data uji
prediction = svm_model.predict(X_test_scaled)
proba = svm_model.predict_proba(X_test_scaled)

print("Prediksi kelas untuk data uji:", prediction[0])
print("Probabilitas prediksi:", proba[0])

# 7. (Opsional) Visualisasi jika hanya 2 fitur
if X_train.shape[1] == 2:
    import numpy as np
    from matplotlib.colors import ListedColormap

    X_combined = np.vstack((X_train_scaled, X_test_scaled))
    y_combined = pd.concat([y_train, pd.Series(prediction)], ignore_index=True)

    x_min, x_max = X_combined[:, 0].min() - 1, X_combined[:, 0].max() + 1
    y_min, y_max = X_combined[:, 1].min() - 1, X_combined[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))

    Z = svm_model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z, alpha=0.3, cmap=ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF']))
    plt.scatter(X_train_scaled[:, 0], X_train_scaled[:, 1], c=y_train, edgecolors='k', label='Train')
    plt.scatter(X_test_scaled[:, 0], X_test_scaled[:, 1], c='black', marker='x', s=100, label='Test')
    plt.title("Visualisasi SVM (2 fitur)")
    plt.xlabel(X_train.columns[0])
    plt.ylabel(X_train.columns[1])
    plt.legend()
    plt.show()

Prediksi kelas untuk data uji: Elevated Proneness
Probabilitas prediksi: [0.13583252 0.26463909 0.14414909 0.34239547 0.11298383]
